In [1]:
import pandas as pd
import pickle
import argparse
import os
import glob
import pydicom
import sys

/tmp/ipykernel_3253249/1002562574.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# 경로만 지정!
base_dir = '/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_simence_SWU1_after'


def calculate_volume(nii_path):
    img = nib.load(nii_path) 
    data = img.get_fdata()
    header = img.header
    voxel_volume = np.prod(header.get_zooms())
    unique_elements, counts_elements = np.unique(data, return_counts=True)
    volumes = counts_elements * voxel_volume
    return dict(zip(unique_elements.astype(int), volumes))

In [3]:
import os
import pandas as pd
import pickle

def load_pickle_file(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)
        
def find_volumes_files(root_path):
    volumes_files = []
    for root, dirs, files in os.walk(root_path):
        for file in files:
            if file == 'volumes.pickle':
                volumes_files.append(os.path.join(root, file))
    return volumes_files

def extract_subject_data(volumes_file_path):
    subject_name = os.path.dirname(volumes_file_path).split(os.sep)[-3]
    volumes_data = load_pickle_file(volumes_file_path)
    volumes_data['subject'] = subject_name
    return volumes_data

def compile_data(volumes_files):
    data = []
    for volumes_file in volumes_files:
        subject_data = extract_subject_data(volumes_file)
        data.append(subject_data)
    df = pd.DataFrame(data)
    # You might need to adjust the DataFrame to ensure it has the structure you want
    return df

root_path = '/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_simence_SWU1_after'
volumes_files = find_volumes_files(root_path)
df = compile_data(volumes_files)

In [4]:
df['subject'] = df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf143', '')


In [5]:
df

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject
0,1.494664e+06,1.437535e+07,228415.007330,6064.239690,554.630972,13579.369306,57620.910056,9345.720523,3521.237820,5531.217717,...,141719.360760,26507.381646,115211.979114,10549.993461,960.742951,11510.736412,903.461954,0.897060,0.009107,0027221_session_2_anat_2
1,1.606571e+06,1.426345e+07,230140.983242,7272.628628,648.955967,15227.827222,59745.794948,8983.512541,3787.062807,6444.283671,...,151379.612266,29905.482472,121474.129794,14902.320239,1289.336934,16191.657173,823.542958,0.870261,0.030113,0027222_session_1_anat_2
2,1.563235e+06,1.430678e+07,254042.251021,3787.748806,499.750974,16964.093133,58736.688999,9971.695491,3820.333805,5479.081720,...,150084.444332,33083.034310,117001.410022,9581.018511,1310.602933,10891.621444,1140.817942,0.876336,0.009574,0027203_session_1_anat_1
3,1.396220e+06,1.447380e+07,219899.689765,3633.741814,408.512979,13998.172285,52245.757331,9138.891533,3384.723827,5308.267729,...,131848.164264,27618.701589,104229.462675,6775.278654,808.793959,7584.072613,1008.762948,0.908463,0.017683,0027206_session_2_anat_1
4,1.500090e+06,1.436993e+07,228406.775331,4520.053769,929.529953,13093.338331,51287.415380,9432.156518,3515.406820,5622.798713,...,127617.945480,25969.557673,101648.387807,7285.319628,1395.323929,8680.643557,885.968955,0.841548,0.092524,0027212_session_1_anat_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,1.548475e+06,1.432154e+07,226906.150407,6317.030677,797.817959,15384.235214,58820.723995,8623.362559,3952.731798,5335.707727,...,147554.819461,30372.648448,117182.171013,14226.953273,1403.555928,15630.509201,701.777964,0.841039,0.012584,0027220_session_2_anat_1
91,1.510540e+06,1.435948e+07,217783.036874,8493.365566,382.787980,13757.729297,50823.679403,8160.655583,3041.723845,4691.896760,...,128228.828449,26711.809635,101517.018814,14771.980245,868.132956,15640.113201,608.824969,0.897057,0.040688,0027210_session_1_anat_1
92,1.311818e+06,1.455820e+07,187929.347399,3674.558812,582.413970,14143.261277,54621.375209,8471.413567,3283.538832,5186.159735,...,136197.404042,27397.123600,108800.280441,7448.244619,1061.241946,8509.486565,850.982957,0.881226,0.008237,0027215_session_1_anat_2
93,1.317193e+06,1.455283e+07,186488.061472,3679.017812,570.065971,14272.229271,54358.980223,8404.528571,3314.751831,5265.735731,...,135855.776059,27527.463594,108328.312466,7458.534619,1045.806947,8504.341566,836.233957,0.883581,0.005275,0027215_session_2_anat_1


In [6]:
root_path1 = "/home/limseoyoung/drwelly1/SeoyoungLim/drwelly-debug/drwelly_simence_SWU3_before"
volumes_files = find_volumes_files(root_path1)
df1 = compile_data(volumes_files)

In [7]:
df1

,total_intracranial_volume,unknown,left_cerebral_white_matter,left_lateral_ventricle,left_inf_lat_vent,left_cerebellum_white_matter,left_cerebellum_cortex,left_thalamus,left_caudate,left_putamen,...,cerebellum,cerebellum_white_matter,cerebellum_cortex,lateral_ventricle,inf_lat_vent,total_lateral_ventricle,wm_hypointensities,HippocampusOccupancyScore,HippocampusAsymmetryIndex,subject
0,1.555781e+06,1.431424e+07,231259.849185,9335.773523,589.273970,13010.675335,57680.935053,8605.183560,3957.533798,5716.094708,...,140102.458842,25888.952677,114213.506165,17519.067105,1025.569948,18544.637053,954.225951,0.888589,0.037663,0027154_session_1_anat_1_Tag_pr100_sr100_ppf10...
1,1.851387e+06,1.401863e+07,261932.965618,33657.216280,1598.722918,15522.121207,63163.446773,10013.541488,4031.621794,6052.234691,...,157197.920969,30215.554456,126982.366513,56993.220088,3015.998846,60009.218934,1676.583914,0.750872,0.035166,0027171_session_1_anat_1_Tag_pr100_sr100_ppf10...
2,1.617235e+06,1.425278e+07,236451.153920,5439.979722,649.984967,14921.871238,60197.182925,8412.760570,4101.936790,6078.302689,...,151350.114268,29112.809513,122237.304755,10433.030467,1206.330938,11639.361405,1039.289947,0.881035,0.024770,0027174_session_2_anat_1_Tag_pr100_sr100_ppf10...
3,1.749064e+06,1.412095e+07,253498.253049,18706.190044,557.031972,15947.098185,67228.682565,9508.302514,4625.011764,5557.971716,...,167563.037439,31227.747405,136335.290035,41261.525892,1619.645917,42881.171809,1088.338944,0.835685,0.005074,0027157_session_2_anat_1_Tag_pr100_sr100_ppf10...
4,1.851387e+06,1.401863e+07,261932.965618,33657.216280,1598.722918,15522.121207,63163.446773,10013.541488,4031.621794,6052.234691,...,157197.920969,30215.554456,126982.366513,56993.220088,3015.998846,60009.218934,1676.583914,0.750872,0.035166,0027171_session_2_anat_1_Tag_pr100_sr100_ppf10...
5,1.455643e+06,1.441438e+07,227633.996370,6938.203646,404.396979,15728.950196,54735.251204,9322.739524,4344.437778,5704.432709,...,140230.054836,30786.992427,109443.062409,12398.077367,721.328963,13119.406330,720.642963,0.918845,0.044002,0027164_session_1_anat_1_Tag_pr100_sr100_ppf10...
6,1.748305e+06,1.412171e+07,253411.817053,18841.332037,569.036971,15843.169191,67539.097549,9529.568513,4610.262764,5556.256716,...,167902.950422,31125.876410,136777.074012,41535.925878,1612.442918,43148.368796,1092.111944,0.837278,0.002641,0027157_session_1_anat_1_Tag_pr100_sr100_ppf10...
7,1.455643e+06,1.441438e+07,227633.996370,6938.203646,404.396979,15728.950196,54735.251204,9322.739524,4344.437778,5704.432709,...,140230.054836,30786.992427,109443.062409,12398.077367,721.328963,13119.406330,720.642963,0.918845,0.044002,0027164_session_2_anat_1_Tag_pr100_sr100_ppf10...
8,1.387557e+06,1.448246e+07,206279.846461,3557.938818,323.448983,13496.363310,55355.738172,8260.468578,3993.205796,5448.897722,...,137914.461954,26661.388638,111253.073316,7098.727637,625.974968,7724.702605,1268.756935,0.934297,0.039397,0027156_session_1_anat_1_Tag_pr100_sr100_ppf10...
9,1.572097e+06,1.429792e+07,247099.588376,4882.604751,706.922964,14369.298266,52890.254298,9192.399530,3949.644798,6529.004666,...,138172.054941,28708.755533,109463.299408,9589.936510,1289.679934,10879.616444,1373.028930,0.877968,0.017626,0027160_session_1_anat_1_Tag_pr100_sr100_ppf10...


In [8]:
df1['subject'] = df1['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf143', '')


In [9]:
combined_df = pd.concat([df, df1], ignore_index=True)

In [10]:
columns_to_include = [
   'whole_brain',	'cortical_gm',	'combined_cerebral_wm',	'frontal','temporal','parietal'	,'occipital','insula'	,'thalamus'	,'caudate','putamen','pallidum','hippocampus','amygdala', 'accumbens_area'	, 'ventraldc' ,'cingulate' ,'cerebellum', 'cerebellum_white_matter' ,'cerebellum_cortex','lateral_ventricle', 'inf_lat_vent',	'total_lateral_ventricle', 	'HippocampusOccupancyScore',	
    'subject'  # 'subject' 컬럼 추가
]

final_df = combined_df[columns_to_include].copy()

In [11]:
sesson1 = final_df[final_df['subject'].str.contains('anat_1')]
sesson2 = final_df[final_df['subject'].str.contains('anat_2')]

In [14]:

all_data = pd.concat([sesson1, sesson2])
all_data_long = pd.melt(all_data, id_vars=["subject"], var_name="ROI", value_name="Volume")

In [15]:
all_data_long

,subject,ROI,Volume
0,0027203_session_1_anat_1,whole_brain,1.290720e+06
1,0027206_session_2_anat_1,whole_brain,1.152243e+06
2,0027215_session_1_anat_1,whole_brain,1.061441e+06
3,0027215_session_3_anat_1,whole_brain,1.055706e+06
4,0027222_session_3_anat_1,whole_brain,1.230997e+06
...,...,...,...
2827,0027216_session_1_anat_2,HippocampusOccupancyScore,9.146385e-01
2828,0027213_session_3_anat_2,HippocampusOccupancyScore,8.831257e-01
2829,0027214_session_3_anat_2,HippocampusOccupancyScore,8.651873e-01
2830,0027215_session_1_anat_2,HippocampusOccupancyScore,8.812263e-01


In [22]:

all_data_long['Session'] = all_data_long['subject'].str.extract(r'(anat_\d+)')


all_data_long['ROI'] = all_data_long['ROI'].str.extract(r'([a-zA-Z-_]+)')[0]

In [23]:

all_data_long['Subject_ID'] = all_data_long['subject'].str.extract(r'(\d+_session_\d+)')

In [29]:
all_data_long

,subject,ROI,Volume,Session,Subject_ID
0,0027203_session_1_anat_1,whole_brain,1.290720e+06,anat_1,0027203_session_1
1,0027206_session_2_anat_1,whole_brain,1.152243e+06,anat_1,0027206_session_2
2,0027215_session_1_anat_1,whole_brain,1.061441e+06,anat_1,0027215_session_1
3,0027215_session_3_anat_1,whole_brain,1.055706e+06,anat_1,0027215_session_3
4,0027222_session_3_anat_1,whole_brain,1.230997e+06,anat_1,0027222_session_3
...,...,...,...,...,...
2827,0027216_session_1_anat_2,HippocampusOccupancyScore,9.146385e-01,anat_2,0027216_session_1
2828,0027213_session_3_anat_2,HippocampusOccupancyScore,8.831257e-01,anat_2,0027213_session_3
2829,0027214_session_3_anat_2,HippocampusOccupancyScore,8.651873e-01,anat_2,0027214_session_3
2830,0027215_session_1_anat_2,HippocampusOccupancyScore,8.812263e-01,anat_2,0027215_session_1


In [24]:

re_long = all_data_long.dropna(subset=['Volume'])


grouped = re_long.groupby(['Subject_ID', 'ROI'])['Volume'].agg(['mean', 'var', 'std']).reset_index()


In [25]:
grouped['CV'] = grouped['std'] / grouped['mean']

In [32]:
grouped = grouped.dropna()
grouped

,Subject_ID,ROI,mean,var,std,CV
552,0027203_session_1,HippocampusOccupancyScore,8.748177e-01,0.000005,0.002147,0.002454
553,0027203_session_1,accumbens_area,1.022483e+03,184.473613,13.582106,0.013283
554,0027203_session_1,amygdala,3.637515e+03,1186.137097,34.440341,0.009468
555,0027203_session_1,caudate,8.009050e+03,376.476762,19.403009,0.002423
556,0027203_session_1,cerebellum,1.498906e+05,75112.996375,274.067503,0.001828
...,...,...,...,...,...,...
1699,0027222_session_3,temporal,1.365080e+05,127633.210519,357.257905,0.002617
1700,0027222_session_3,thalamus,1.743160e+04,39.765358,6.305978,0.000362
1701,0027222_session_3,total_lateral_ventricle,1.608464e+04,211.768178,14.552257,0.000905
1702,0027222_session_3,ventraldc,9.377963e+03,211.768178,14.552257,0.001552


In [26]:
roi_mean_cv = grouped.groupby('ROI')['CV'].mean().reset_index()
roi_mean_cv.rename(columns={'CV': 'Mean_CV'}, inplace=True)

roi_mean_cv

,ROI,Mean_CV
0,HippocampusOccupancyScore,0.002515
1,accumbens_area,0.011383
2,amygdala,0.008570
3,caudate,0.003324
4,cerebellum,0.001994
5,cerebellum_cortex,0.002312
6,cerebellum_white_matter,0.005346
7,cingulate,0.003409
8,combined_cerebral_wm,0.001440
9,cortical_gm,0.002611


In [36]:
grouped.to_csv("drwelly_scan_repetition_after_simen.csv")

In [37]:
grouped

,Subject_ID,ROI,mean,var,std,CV
552,0027203_session_1,HippocampusOccupancyScore,8.748177e-01,0.000005,0.002147,0.002454
553,0027203_session_1,accumbens_area,1.022483e+03,184.473613,13.582106,0.013283
554,0027203_session_1,amygdala,3.637515e+03,1186.137097,34.440341,0.009468
555,0027203_session_1,caudate,8.009050e+03,376.476762,19.403009,0.002423
556,0027203_session_1,cerebellum,1.498906e+05,75112.996375,274.067503,0.001828
...,...,...,...,...,...,...
1699,0027222_session_3,temporal,1.365080e+05,127633.210519,357.257905,0.002617
1700,0027222_session_3,thalamus,1.743160e+04,39.765358,6.305978,0.000362
1701,0027222_session_3,total_lateral_ventricle,1.608464e+04,211.768178,14.552257,0.000905
1702,0027222_session_3,ventraldc,9.377963e+03,211.768178,14.552257,0.001552


In [38]:
pwd

'/home/limseoyoung/debug'